In the next sections of this notebook, we look at the algorithm implementation. To start with, let’s look at the quantum register. The quantum register has a constructor and GetMeasure and ApplyOperation methods. The GetMeasure method computes the quantum states and probabilities. The ApplyOperation method updates the class instance variable data to the product of the input gate matrix and data instance.

In [ ]:
import numpy as nump
from itertools import product

class Quantum_Gate:
    def __init__(self, matrix):
        self._data = nump.array(matrix, dtype=nump.complex64)
        assert len(self._data.shape) == 2
        assert self._data.shape[0] == self._data.shape[1]
        self._n = nump.log2(self._data.shape[0])
        assert self._n.is_integer()
        self._n = int(self._n)
    def __matmul__(self, other):
        return Quantum_Gate(nump.kron(self._data, other._data))
    def __pow__(self, n, modulo=None):
        x = self._data.copy()
        for _ in range(n - 1):
            x = nump.kron(x, self._data)
        return Quantum_Gate(x)

I_Gate = Quantum_Gate([[1, 0], [0, 1]])
H_Gate = Quantum_Gate(nump.array([[1, 1], [1, -1]]) / nump.sqrt(2))
X_Gate = Quantum_Gate([[0, 1], [1, 0]])
Y_Gate = Quantum_Gate([[0, -1j], [1j, 0]])
Z_Gate = Quantum_Gate([[1, 0], [0, -1]])

def U_Quantum_Gate(f, n):
    m = n + 1
    U = nump.zeros((2**m, 2**m), dtype=nump.complex64)
    def bin2int(xs):
        r = 0
        for i, x in enumerate(reversed(xs)):
            r += x * 2 ** i
        return r

    for xs in product({0, 1}, repeat=m):
        x = xs[:~0]
        y = xs[~0]
        z = y ^ f(*x)
        instate = bin2int(xs)
        outstate = bin2int(list(x) + [z])
        U[instate, outstate] = 1
    return Quantum_Gate(U)

In [ ]:
import numpy as nump
from itertools import product
#from Quantum_Gate import Quantum_Gate

class Quantum_Register:
    def __init__(self, n_qbits, init):
        self._n = n_qbits
        assert len(init) == self._n
        self._data = nump.zeros((2 ** self._n), dtype=nump.complex64)
        self._data[int('0b' + init, 2)] = 1
    
    def Get_Measure(self):
        probs = nump.real(self._data) ** 2 + nump.imag(self._data) ** 2
        states = nump.arange(2 ** self._n)
        mstate = nump.random.choice(states, size=1, p=probs)[0]
        return f'{mstate:>0{self._n}b}'

    def Apply_Gate(self, gate):
        assert isinstance(gate, Quantum_Gate)
        assert self._n == gate._n
        self._data = gate._data @ self._data

**Deutsch-Jozsa Algorithm:** 

The algorithm identifies the black-box oracle function. This algorithm uses a black-box (Oracle) function that takes as input the binary values and gives either 0 or 1 as output. The method finds whether the function g used is constant or balanced.

In [ ]:

# Deutsch_Jozsa_Algorithm.py
#from Quantum_Register import Quantum_Register
#from Quantum_Gate import H_Gate, I_Gate, U_Gate

def Check_If_Constant(g, n):
    qr = Quantum_Register(n + 1, '0' * n + '1')
    qr.Apply_Gate(H_Gate ** (n + 1))
    qr.Apply_Gate(U_Gate(g, n))
    qr.Apply_Gate(H_Gate ** n @ I_Gate)
    return qr.Get_Measure()[:~0] == '0' * n

def g1_func(v):
    return v

def g2_func(w):
    return 1

def g3_func(v, w):
    return v ^ w

def g4_func(v, w,x):
    return 0

print('g(v) = v is {}'.format('constant function' if Check_If_Constant(g1_func, 1) else 'balanced function'))
print('g(v) = 1 is {}'.format('constant function' if Check_If_Constant(g2_func, 1) else 'balanced function'))
print('g(v, w) = v ^ w is {}'.format('constant function' if Check_If_Constant(g3_func, 2) else 'balanced function'))
print('g(v, w, x) = 0 is {}'.format('constant function' if Check_If_Constant(g4_func, 3) else 'balanced function'))

**Simon’s Algorithm**

This algorithm helps in identifying a black-box function h(x), where h(x) = h(y) and x = y ⊕ t. Note that t ∈ {0, 1}n and ⊕ represents bitwise addition based on module 2. The goal of the algorithm is to find that by using h. This quantum algorithm performs exponentially better than the classical equivalent.

In [ ]:


from collections import defaultdict
import numpy as nump
from mock import patch
from collections import defaultdict
from operator import xor
from typing import Dict, Tuple, List
import numpy.random as rand
from pyquil import Program
from pyquil.api import QuantumComputer
from pyquil.gates import H, MEASURE

def RetrieveOnetoOneMap(mask: str) -> Dict[str, str]:
    n_bits = len(mask)
    form_string = "{0:0" + str(n_bits) + "b}"
    bit_map_dct = {}
    for idx in range(2**n_bits):
        bit_string = form_string.format(idx)
        bit_map_dct[bit_string] = InvokeBitwiseXorOperation(bit_string, mask)
    return bit_map_dct

def RetrieveValidTwotoOneMap(mask: str, random_seed: int = None) -> Dict[str, str]:
    if random_seed is not None:
        rand.seed(random_seed)
    bit_map = RetrieveOnetoOneMap(mask)
    n_samples = int(len(bit_map.keys()) / 2)
    range_of_2to1_map = list(rand.choice(list(sorted(bit_map.keys())), replace=False, size=n_samples))
    list_of_bitstring_tuples = sorted([(k, v) for k, v in bit_map.items()], key=lambda x: x[0])
    bit_map_dct = {}
    for cnt in range(n_samples):
        bitstring_tup = list_of_bitstring_tuples[cnt]
        val = range_of_2to1_map[cnt]
        bit_map_dct[bitstring_tup[0]] = val
        bit_map_dct[bitstring_tup[1]] = val
    return bit_map_dct

class Simons_Algorithm(object):
    def __init__(self):
        self.unitary_function_mapping = None
        self.n_qubits = None
        self.n_ancillas = None
        self._qubits = None
        self.computational_qubits = None
        self.ancillas = None
        self.simon_circuit = None
        self._dict_of_linearly_indep_bit_vectors = {}
        self.search_mask = None
        self.bit_map = None
        self.classical_register = None

    def RetrieveSimonCircuit(self) -> Program:
        simon_circuit = Program()
        oracle_name = "SIMON_ORACLE_FUNCTION"
        simon_circuit.defgate(oracle_name, self.unitary_function_mapping)
        simon_circuit.inst([H(i) for i in self.computational_qubits])
        simon_circuit.inst(tuple([oracle_name] + sorted(self._qubits, reverse=True)))
        simon_circuit.inst([H(i) for i in self.computational_qubits])
        return simon_circuit

    def _Initialize_Attributes(self, bitstring_map: Dict[str, str]) -> None:
        self.bit_map = bitstring_map
        self.n_qubits = len(list(bitstring_map.keys())[0])
        self.n_ancillas = self.n_qubits
        self._qubits = list(range(self.n_qubits + self.n_ancillas))
        self.computational_qubits = self._qubits[:self.n_qubits]
        self.ancillas = self._qubits[self.n_qubits:]
        self.unitary_function_mapping, _ = self.CalculateUnitaryOracle(bitstring_map)
        self.simon_circuit = self.RetrieveSimonCircuit()
        self._dict_of_linearly_indep_bit_vectors = {}
        self.search_mask = None
    @staticmethod
    def CalculateUnitaryOracle(bitstring_map: Dict[str, str]) -> Tuple[nump.ndarray,Dict[str, str]]:
        n_bits = len(list(bitstring_map.keys())[0])
        ufunc = nump.zeros(shape=(2 ** (2 * n_bits), 2 ** (2 * n_bits)))
        index_mapping_dct = defaultdict(dict)
        for b in range(2**n_bits):
            pad_str = nump.binary_repr(b, n_bits)
            for k, v in bitstring_map.items():
                index_mapping_dct[pad_str + k] = InvokeBitwiseXorOperation(pad_str, v) + k
                i, j = int(pad_str+k, 2), int(InvokeBitwiseXorOperation(pad_str, v) + k, 2)
                ufunc[i, j] = 1
        self._Initialize_Attributes(bitstring_map)
        self.RetrieveSampleIndependentBits(qc)
        self.RetrieveInverseMaskEquation()
        if self.CheckValidMaskIfCorrect():
            return self.search_mask
        else:
            raise Exception("No valid mask found")

    def RetrieveSampleIndependentBits(self, quantum_computer: QuantumComputer) -> None:
        while len(self._dict_of_linearly_indep_bit_vectors) < self.n_qubits - 1:
            prog = Program()
            simon_ro = prog.declare('ro', 'BIT', len(self.computational_qubits))
            prog += self.simon_circuit
            prog += [MEASURE(qubit, ro) for qubit, ro in zip(self.computational_qubits, simon_ro)]
            executable = quantum_computer.compile(prog)
            sampled_bit_string = nump.array(quantum_computer.run(executable)[0], dtype=int)
            self.AppendToDictofIndepBits(sampled_bit_string)

    def RetrieveInverseMaskEquation(self) -> None:
        missing_msb = self.RetrieveMissingMsbVector()
        upper_triangular_matrix = nump.asarray(
            [tup[1] for tup in sorted(zip(self._dict_of_linearly_indep_bit_vectors.keys(),
                                          self._dict_of_linearly_indep_bit_vectors.values()),
                                      key=lambda x: x[0])])
        msb_unit_vec = nump.zeros(shape=(self.n_qubits,), dtype=int)
        msb_unit_vec[missing_msb] = 1
        self.search_mask = RetrieveBinaryBackSubstitute(upper_triangular_matrix, msb_unit_vec).tolist()

    def AppendToDictofIndepBits(self, z: nump.ndarray) -> None:
        if (z == 0).all() or (z == 1).all():
            return None
        msb_z = RetrieveMostSignificantBit(z)
        if msb_z not in self._dict_of_linearly_indep_bit_vectors.keys():
            self._dict_of_linearly_indep_bit_vectors[msb_z] = z
        else:
            conflict_z = self._dict_of_linearly_indep_bit_vectors[msb_z]
            not_z = [xor(conflict_z[idx], z[idx]) for idx in range(len(z))]
            if (nump.asarray(not_z) == 0).all():
                return None
            msb_not_z = most_significant_bit(nump.asarray(not_z))
            if msb_not_z not in self._dict_of_linearly_indep_bit_vectors.keys():
                self._dict_of_linearly_indep_bit_vectors[msb_not_z] = not_z

    def RetrieveMissingMsbVector(self) -> int:
        missing_msb = None
        for idx in range(self.n_qubits):
            if idx not in self._dict_of_linearly_indep_bit_vectors.keys():
                missing_msb = idx
        if missing_msb is None:
            raise ValueError("Expected a missing provenance, but didn't find one.")
        augment_vec = nump.zeros(shape=(self.n_qubits,))
        augment_vec[missing_msb] = 1
        self._dict_of_linearly_indep_bit_vectors[missing_msb] = augment_vec.astype(int).tolist()
        return missing_msb

    def CheckValidMaskIfCorrect(self) -> bool:
        mask_str = ''.join([str(b) for b in self.search_mask])
        return all([self.bit_map[k] == self.bit_map[InvokeBitwiseXorOperation(k, mask_str)]
                    for k in self.bit_map.keys()])

PADDED_BINARY_BIT_STRING = "{0:0{1:0d}b}"

def CheckValidIfUnitary(matrix: nump.ndarray) -> bool:
    rows, cols = matrix.shape
    if rows != cols:
        return False
    return nump.allclose(nump.eye(rows), matrix.dot(matrix.T.conj()))

def RetrieveMostSignificantBit(lst: nump.ndarray) -> int:
    return nump.argwhere(nump.asarray(lst) == 1)[0][0]

def InvokeBitwiseXorOperation(bs0: str, bs1: str) -> str:
    if len(bs0) != len(bs1):
        raise ValueError("Bit strings are not of equal length")
    n_bits = len(bs0)
    return PADDED_BINARY_BIT_STRING.format(xor(int(bs0, 2), int(bs1, 2)), n_bits)

def RetrieveBinaryBackSubstitute(W: nump.ndarray, s: nump.ndarray) -> nump.ndarray:
    m = nump.copy(s)
    n = len(s)
    for row_num in range(n - 2, -1, -1):
        row = W[row_num]
        for col_num in range(row_num + 1, n):
            if row[col_num] == 1:
                m[row_num] = xor(s[row_num], s[col_num])
    return m[::-1]


search_mask = '110'
bm = RetrieveValidTwotoOneMap(search_mask, random_seed=42)
expected_map = {
    '000': '001',
    '001': '101',
    '010': '000',
    '011': '111',
    '100': '000',
    '101': '111',
    '110': '001',
    '111': '101'}

for k, v in bm.items():
    assert v == expected_map[k]

reverse_bitmap = defaultdict(list)
for k, v in bm.items():
    reverse_bitmap[v].append(k)

expected_reverse_bitmap = {
    '001': ['000', '110'],
    '101': ['001', '111'],
    '000': ['010', '100'],
    '111': ['011', '101']}

for k, v in reverse_bitmap.items():
    assert sorted(v) == sorted(expected_reverse_bitmap[k])



with patch("pyquil.api.QuantumComputer") as quantum_computer:
    quantum_computer.run.side_effect = [
        (nump.asarray([0, 1, 1], dtype=int), ),
        (nump.asarray([1, 1, 1], dtype=int), ),
        (nump.asarray([1, 1, 1], dtype=int), ),
        (nump.asarray([1, 0, 0], dtype=int), ),
    ]

simon_algo = Simons_Algorithm()
result_mask = simon_algo.RetrieveBitMask(quantum_computer, bm)
print("mask", search_mask," result mask",result_mask)

**Shor's Algorithm**

Shor’s algorithm is used for prime factorization. The QuantumMap class has the properties of state and amplitude. QuantumEntanglement has the properties of amplitude, register, and entangled. The UpdateEntangled method of the QuantumEntanglement class takes input parameters such as state and amplitude. The RetrieveEntangles method returns the list of entangled states.6

In [ ]:
import math
import random



class QuantumMap:
    def __init__(self, state, amplitude):
        self.state = state
        self.amplitude = amplitude


class QuantumEntanglement:
    def __init__(self, amplitude, register):
        self.amplitude = amplitude
        self.register = register
        self.entangled = {}

    def UpdateEntangled(self, fromState, amplitude):
        register = fromState.register
        entanglement = QuantumMap(fromState, amplitude)
        try:
            self.entangled[register].append(entanglement)
        except KeyError:
            self.entangled[register] = [entanglement]

    def RetrieveEntangles(self, register = None):
        entangles = 0
        if register is None:
            for states in self.entangled.values():
                entangles += len(states)
        else:
            entangles = len(self.entangled[register])

        return entangles


class QuantumRecord:
    def __init__(self, numBits):
        self.numBits = numBits
        self.numStates = 1 << numBits
        self.entangled = []
        self.states = [QuantumEntanglement(complex(0.0), self) for x in range(self.numStates)]
        self.states[0].amplitude = complex(1.0)

    def UpdatePropagate(self, fromRegister = None):
        if fromRegister is not None:
            for state in self.states:
                amplitude = complex(0.0)

                try:
                    entangles = state.entangled[fromRegister]
                    for entangle in entangles:
                        amplitude += entangle.state.amplitude * entangle.amplitude

                    state.amplitude = amplitude
                except KeyError:
                    state.amplitude = amplitude

        for register in self.entangled:
            if register is fromRegister:
                continue

            register.UpdatePropagate(self)

    def UpdateMap(self, toRegister, mapping, propagate = True):
        self.entangled.append(toRegister)
        toRegister.entangled.append(self)

        mapTensorX = {}
        mapTensorY = {}
        for x in range(self.numStates):
            mapTensorX[x] = {}
            codomain = mapping(x)
            for element in codomain:
                y = element.state
                mapTensorX[x][y] = element

                try:
                    mapTensorY[y][x] = element
                except KeyError:
                    mapTensorY[y] = { x: element }

        def UpdateNormalize(tensor, p = False):
            lSqrt = math.sqrt
            for vectors in tensor.values():
                sumProb = 0.0
                for element in vectors.values():
                    amplitude = element.amplitude
                    sumProb += (amplitude * amplitude.conjugate()).real

                normalized = lSqrt(sumProb)
                for element in vectors.values():
                    element.amplitude = element.amplitude / normalized

        UpdateNormalize(mapTensorX)
        UpdateNormalize(mapTensorY, True)

        for x, yStates in mapTensorX.items():
            for y, element in yStates.items():
                amplitude = element.amplitude
                toState = toRegister.states[y]
                fromState = self.states[x]
                toState.UpdateEntangled(fromState, amplitude)
                fromState.UpdateEntangled(toState, amplitude.conjugate())

        if propagate:
            toRegister.UpdatePropagate(self)

    def RetrieveMeasure(self):
        measure = random.random()
        sumProb = 0.0

        # Pick a state
        finalX = None
        finalState = None
        for x, state in enumerate(self.states):
            amplitude = state.amplitude
            sumProb += (amplitude * amplitude.conjugate()).real

            if sumProb > measure:
                finalState = state
                finalX = x
                break

        if finalState is not None:
            for state in self.states:
                state.amplitude = complex(0.0)

            finalState.amplitude = complex(1.0)
            self.UpdatePropagate()

        return finalX

    def RetrieveEntangles(self, register = None):
        entangles = 0
        for state in self.states:
            entangles += state.entangles(None)

        return entangles

    def RetrieveAmplitudes(self):
        amplitudes = []
        for state in self.states:
            amplitudes.append(state.amplitude)

        return amplitudes

def FindListEntangles(register):
    print("Entangles: " + str(register.RetrieveEntangles()))

def FindListAmplitudes(register):
    amplitudes = register.amplitudes()
    for x, amplitude in enumerate(amplitudes):
        print('State #' + str(x) + '\'s Amplitude value: ' + str(amplitude))

def InvokeHadamard(x, Q):
    codomain = []
    for y in range(Q):
        amplitude = complex(pow(-1.0, RetrieveBitCount(x & y) & 1))
        codomain.append(QuantumMap(y, amplitude))

    return  codomain

def InvokeQModExp(a, exp, mod):
    state = InvokeModExp(a, exp, mod)
    amplitude = complex(1.0)
    return [QuantumMap(state, amplitude)]

def InvokeQft(x, Q):
    fQ = float(Q)
    k = -2.0 * math.pi
    codomain = []

    for y in range(Q):
        theta = (k * float((x * y) % Q)) / fQ
        amplitude = complex(math.cos(theta), math.sin(theta))
        codomain.append(QuantumMap(y, amplitude))

    return codomain

def DeterminePeriod(a, N):
    nNumBits = N.bit_length()
    inputNumBits = (2 * nNumBits) - 1
    inputNumBits += 1 if ((1 << inputNumBits) < (N * N)) else 0
    Q = 1 << inputNumBits

    print("The period is...")
    print("Q = " + str(Q) + "\ta = " + str(a))
    
    inputRegister = QuantumRecord(inputNumBits)
    hmdInputRegister = QuantumRecord(inputNumBits)
    qftInputRegister = QuantumRecord(inputNumBits)
    outputRegister = QuantumRecord(inputNumBits)

    print("Registers are instantiated")
    print("Executing Hadamard on the input")

    inputRegister.UpdateMap(hmdInputRegister, lambda x: InvokeHadamard(x, Q), False)

    print("Hadamard operation is invoked")
    print("Mapping input register to the output")

    hmdInputRegister.UpdateMap(outputRegister, lambda x: InvokeQModExp(a, x, N), False)

    print("Modular exponentiation is invoked")
    print("Executing quantum Fourier transform on the output")

    hmdInputRegister.UpdateMap(qftInputRegister, lambda x: InvokeQft(x, Q), False)
    inputRegister.UpdatePropagate()

    print("Quantum Fourier transform is invoked")
    print("Retrieving a measurement on the output")

    y = outputRegister.RetrieveMeasure()

    print("Measuring the Output register \ty = " + str(y))


    print("Retrieving  a measurement on the periodicity")

    x = qftInputRegister.RetrieveMeasure()

    print("Measuring QFT  \tx = " + str(x))

    if x is None:
        return None

    print("Retrieving the period via continued fractions")

    r = RetrieveContinuedFraction(x, Q, N)

    print("Determined Candidate period\tr = " + str(r))

    return r



def RetrieveBitCount(x):
    sumBits = 0
    while x > 0:
        sumBits += x & 1
        x >>= 1

    return sumBits

def RetrieveGcd(a, b):
    while b != 0:
        tA = a % b
        a = b
        b = tA

    return a

def RetrieveExtendedGCD(a, b):
    fractions = []
    while b != 0:
        fractions.append(a // b)
        tA = a % b
        a = b
        b = tA

    return fractions

def RetrieveContinuedFraction(y, Q, N):
    fractions = RetrieveExtendedGCD(y, Q)
    depth = 2

    def RetrievePartial(fractions, depth):
        c = 0
        r = 1

        for i in reversed(range(depth)):
            tR = fractions[i] * r + c
            c = r
            r = tR

        return c

    r = 0
    for d in range(depth, len(fractions) + 1):
        tR = RetrievePartial(fractions, d)
        if tR == r or tR >= N:
            return r

        r = tR

    return r

def InvokeModExp(a, exp, mod):
    fx = 1
    while exp > 0:
        if (exp & 1) == 1:
            fx = fx * a % mod
        a = (a * a) % mod
        exp = exp >> 1

    return fx

def RetrieveRandom(N):
    a = math.floor((random.random() * (N - 1)) + 0.5)
    return a

def RetrieveNeighBorCandidates(a, r, N, neighborhood):
    if r is None:
        return None

    for k in range(1, neighborhood + 2):
        tR = k * r
        if InvokeModExp(a, a, N) == InvokeModExp(a, a + tR, N):
            return tR

    for tR in range(r - neighborhood, r):
        if InvokeModExp(a, a, N) == InvokeModExp(a, a + tR, N):
            return tR

    for tR in range(r + 1, r + neighborhood + 1):
        if InvokeModExp(a, a, N) == InvokeModExp(a, a + tR, N):
            return tR

    return None

def ExecuteShorsAlgorithm(N, attempts = 1, neighborhood = 0.0, numPeriods = 1):
    
    periods = []
    neighborhood = math.floor(N * neighborhood) + 1

    print("N value is" + str(N))
    print("Neighborhood value is = " + str(neighborhood))
    print("Number of periods is = " + str(numPeriods))

    for attempt in range(attempts):
        print("\nAttempt #" + str(attempt))

        a = RetrieveRandom(N)
        while a < 2:
            a = RetrieveRandom(N)

        d = RetrieveGcd(a, N)
        if d > 1:
            print("Determined factors classically, re-attempt")
            continue

        r = DeterminePeriod(a, N)

        print("validating the candidate period, nearby values, and multiples")

        r = RetrieveNeighBorCandidates(a, r, N, neighborhood)

        if r is None:
            print("Period was not determined, re-attempt")
            continue

        if (r % 2) > 0:
            print("Period is odd, re-attempt")
            continue

        d = InvokeModExp(a, (r // 2), N)
        if r == 0 or d == (N - 1):
            print("Period is trivial, re-attempt")
            continue

        print("Period found\tr = " + str(r))

        periods.append(r)
        if(len(periods) < numPeriods):
            continue

        print("\n Determining  least common multiple of all periods")

        r = 1
        for period in periods:
            d = RetrieveGcd(period, r)
            r = (r * period) // d

        b = InvokeModExp(a, (r // 2), N)
        f1 = RetrieveGcd(N, b + 1)
        f2 = RetrieveGcd(N, b - 1)

        return [f1, f2]

    return None


results = ExecuteShorsAlgorithm(35, 20, 0.01, 2)
print("Results are:\t" + str(results[0]) + ", " + str(results[1]))

**Grover’s Algorithm** 

Grover’s algorithm is used for searching a database. The search algorithm can find an entry in a database of M entries with O (√N) time and O(logN) space. This method was found by Lov Grover in 1996. He found that his method gave a quadratic speedup compared to other quantum algorithms.

Grover’s algorithm consists of the following steps: 

1. Initialize the system to the state.
 
2. Execute the Grover iteration r(M) ties.

3. Measure the amplitude Ω. 

4. From Ωw (result), find w. Let’s look

In [ ]:


import matplotlib.pyplot as plot
import numpy as nump
import string
import hashlib
from math import sqrt, pi
from collections import OrderedDict
from statistics import mean

def RenderGraph(amplitude_value, n):
    y_position = nump.arange(n)
    plot.bar(y_position, amplitude_value.values(), align='center', color='g')
    plot.xticks(y_position, amplitude_value.keys())
    plot.ylabel('Amplitude Value')
    plot.title('Grovers Algorithm')
    plot.show()
    
def ApplyOracleFunction(xvalue):
    return hashlib.sha256(bytes(xvalue, 'utf-8')).hexdigest()

def ApplyGroverAlgorithm(target, objects, nvalue, rounds):
    y_pos = nump.arange(nvalue)
    amplitude = OrderedDict.fromkeys(objects, 1/sqrt(nvalue))

    for i in range(0, rounds, 2):
        for k, v in amplitude.items():
            if ApplyOracleFunction(k) == target:
                amplitude[k] = v * -1

        average = mean(amplitude.values())
        for k, v in amplitude.items():
            if ApplyOracleFunction(k) == target:
                amplitude[k] = (2 * average) + abs(v)
                continue
            amplitude[k] = v-(2*(v-average))
    return amplitude

target_algorithm = '9'
objects_grover = ('14', '5', '13', '7','9','11','97')
number = len(objects_grover)
amplitude_grover = OrderedDict.fromkeys(objects_grover, 1/sqrt(number))

amplitude_grover[target_algorithm] = amplitude_grover[target_algorithm] * -1
print(amplitude_grover)
average_grover = mean(amplitude_grover.values())
print("Mean is {}".format(average_grover))
for k, v in amplitude_grover.items():
    if k == target_algorithm:
        amplitude_grover[k] = (2 * average_grover) + abs(v)
        continue
    amplitude_grover[k] = v-(2*(v-average_grover))
print(amplitude_grover)

needle_value = "2d711642b726b04401627ca9fbac32f5c8530fb1903cc4db02258717921a4881"
haystack_value = string.ascii_lowercase
num = len(haystack_value)
num_rounds = int((pi / 4) * sqrt(num))
print("number of rounds are {}".format(num_rounds))
RenderGraph(ApplyGroverAlgorithm(needle_value, haystack_value, num, num_rounds), num)